# <span style="color:darkred">Multiplicative Schwarz  using multiple strategies</span>.

<img align="left" src="Schwarz.jpg" width=120 height=120 />

### [Simon Tavener](https://www.math.colostate.edu/~tavener/) and Andre Leautaud

#### Full text for this notebook can be found at the following [link](https://www.math.colostate.edu/~tavener/andre/DD_multiphysics.pdf)

This notebook illustrates three ways of solving a system using Multiplicative Schwarz. It first creates partitions and necessary matrices, then steps through the forms of iteration. It demonstrates C Iteration (each subdomain independently), D Iteration (subdomains calculated in parallel) and  U Iteration (computes all iterations as one matrix solve). In each, we keep track of total error, iteration error, and discretization error (true and estimates). If the estimates are good, and the system was solved, the effectivty ratio will be 1. It can read all parameters and the linear system from a csv. It can also create a csv. This csv is compatible with Multiplicative Schwarz and the MatLab Live files.

#### Set path and parameters

In [37]:
# load necessary packages
import numpy as np
import math
%run DD_Utilities.ipynb
%run MS_Utilities.ipynb
iprint= 4 # amount of output desired

In [38]:
iparam= -2

In [39]:
if iparam == -2:
    n= 12
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,12] #List of ends (inclusive) or integer representing overlap
    K= 3
    alpha= 0.5
    mpert= 0.0001
    if K > 500:
        raise ValueError("K is too large")

In [40]:
if iparam == 0:
    n= 10  # size of system
    p= 4   # number of subomains
    b= [1,2,4,7]  # indices of starts of subdomains (1 indexed), should be in ascending order.
    e= [3,4,8,10] # indices of ends of subdomains (1 indexed), need not be in ascending order. 
                  # Setting e to an integer will make each subdomain overlap the next by that number.

    K= 3  # number of iterations
    alpha= 0.5 # alpha is not used in Multiplicative Schwarz. It is here so that the parameters in the csv are compatible with Additive Schwarz.
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [41]:
# This example shows how to create domains that each overlap by 1 column
#small system
if iparam == 1:
    n= 8
    p= 3 
    b= [1,3,5]
    e= 1 # causes each subdomain to overlap the previous by 1
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [42]:
#subdomains embedded in others
if iparam == 2:
    n= 10
    p= 5
    b= [1,1,5,5,7]
    e= [2,4,6,8,10] 
    K= 6
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [43]:
#overlap does not connect all subdomains
if iparam == 3:
    n= 9
    p= 4
    b= [1,3,6,8]
    e= [2,5,7,9]
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [44]:
if iparam == 4:
    n= 12
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,12]
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [45]:
print(p,b,e)
nn,nb,ne,index= fix_domains(n,p,b,e,iprint)

4 [1, 2, 5, 9] [3, 7, 10, 12]


#### Establish and solve forward system 

In [46]:
# Read and write capability. Usually only use one of sread and swrite, but it is acceptable to set both to True.
sread= False   #Read system of equations from file_name. The solver will then ignore all of the parameters set above.
swrite= True  #Write system to file_name file.
file_name= 'iparam' + str(iparam) + '.csv'

M,x,y,xstore= Linear_system(n,sread,file_name)
wpert= Make_wpert(n,p,K,sread,iparam) #Read or create wpert (depending on sread)

if iprint >= 4:
    print(M)
    print('y')
    print(y)        
    print('x')
    print(x)

if swrite:
    save_to_csv(n,p,K,alpha,nb,nn,ne,M,y,mpert,wpert,iparam)

[[1.24170220e+01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
  3.96767474e-01 5.38816734e-01 4.19194514e-01 6.85219500e-01]
 [2.04452250e-01 1.28781174e+01 2.73875932e-02 6.70467510e-01
  4.17304802e-01 5.58689828e-01 1.40386939e-01 1.98101489e-01
  8.00744569e-01 9.68261576e-01 3.13424178e-01 6.92322616e-01]
 [8.76389152e-01 8.94606664e-01 1.20850442e+01 3.90547832e-02
  1.69830420e-01 8.78142503e-01 9.83468338e-02 4.21107625e-01
  9.57889530e-01 5.33165285e-01 6.91877114e-01 3.15515631e-01]
 [6.86500928e-01 8.34625672e-01 1.82882773e-02 1.27501443e+01
  9.88861089e-01 7.48165654e-01 2.80443992e-01 7.89279328e-01
  1.03226007e-01 4.47893526e-01 9.08595503e-01 2.93614148e-01]
 [2.87775339e-01 1.30028572e-01 1.93669579e-02 6.78835533e-01
  1.22116281e+01 2.65546659e-01 4.91573159e-01 5.33625451e-02
  5.74117605e-01 1.46728575e-01 5.89305537e-01 6.99758360e-01]
 [1.02334429e-01 4.14055988e-01 6.94400158e-01 4.14179270e-01
  4

### Iterative strategy
$u^{\{k\}}= u^{\{k-1\}} + Br = u^{\{k-1\}} + B(f-Au^{\{k-1\}})$, $\quad k=1, \dots, K,$ where $B$ is an approximation to $A^{-1}$

<ins>Multiplicative<ins>
$A_i= R_i A R_i^\top \in \mathbb{R}^{m_i \times m_i}, \\
B_i= R_i^\top A_i^{-1} R_i  = R_i^\top ( R_i A R_i^\top)^{-1} R_i  \in \mathbb{R}^{n \times n}, \\
C_i= B_i A   \in \mathbb{R}^{n \times n},  \\
D_i= (I-C_i) \in \mathbb{R}^{n \times n},  \\
f_i= B_i f   \in \mathbb{R}^{n}, \; i=1,\dots,p.
$

In [47]:
#Construct domains, R,A,B,C and f
R= Rmatrices(n,p,nn,index,iprint)
A= Amatrices(M,R,n,p,nn,iprint)
B= Bmatrices(A,R,n,p,nn,iprint)
C= Cmatrices(M,B,n,p,iprint) 
D= Dmatrices_multiplicative(C,n,p,iprint)
f= fvector(B,y,n,p,iprint)

Print DD matrix 
[[[ 1.11022302e-16  5.95418562e-18  1.16740496e-20 -2.13474482e-02
   -9.94954069e-03 -4.93219822e-03 -1.43819684e-02 -2.69656891e-02
   -2.83808929e-02 -3.90718100e-02 -3.23837005e-02 -5.21153314e-02]
  [-3.30988314e-18  0.00000000e+00 -8.95397514e-20 -5.17281927e-02
   -3.22229403e-02 -4.31576001e-02 -1.06594625e-02 -1.48870906e-02
   -6.15736291e-02 -7.44902042e-02 -2.37105842e-02 -5.28930746e-02]
  [-1.03439791e-17 -4.47790896e-18  0.00000000e+00  2.14564996e-03
   -1.09460827e-02 -6.91111168e-02 -6.30586080e-03 -3.17878106e-02
   -7.26462088e-02 -3.57701415e-02 -5.31470744e-02 -1.84131575e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.000

#### Perform $K$  iterations computing subdomain contributions independently
$
u^{\{k+i/p\}} = u^{\{k+(i-1)/p\}} + B_i(f-Au^{\{k+(i-1)/p\}}) =D_i u^{\{k+(i-1)/p\}} + f_i \quad i=1,\dots,p$

In [48]:
#Perform D iteration
vstore= Diteration_multiplicative(D,f,n,p,K,iprint)
if iprint >= 4:
    print('vstore')
    print(vstore)


Outer iteration k, Inner iteration i 0 0
[[0.00310865 0.00825975 0.01783988 0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]]
Outer iteration k, Inner iteration i 0 1
[[ 0.00310865  0.00423357  0.01751076  0.05223098  0.04278771 -0.00208609
   0.00106365  0.          0.          0.          0.          0.        ]]
Outer iteration k, Inner iteration i 0 2
[[ 0.00310865  0.00423357  0.01751076  0.05223098  0.04099262 -0.00765256
  -0.00540388  0.06658251  0.03862675  0.00578671  0.          0.        ]]
Outer iteration k, Inner iteration i 0 3
[[ 0.00310865  0.00423357  0.01751076  0.05223098  0.04099262 -0.00765256
  -0.00540388  0.06658251  0.03496864  0.00507412  0.00788476  0.05114969]]
Outer iteration  0
[[ 0.00310865  0.00423357  0.01751076  0.05223098  0.04099262 -0.00765256
  -0.00540388  0.06658251  0.03496864  0.00507412  0.00788476  0.05114969]]
Outer iteration k, Inner iteration i 1 0
[[-0.00420591 -0.00178986  0.01186696  0.05223

#### Perform $K$ iterations using iteration matrices $S$ and $T$,  and  which compute the solution in all $p$ domains simultanously
Let $p=5$ and construct S and T, g and v such that

$S = \pmatrix{
 I   &  0   &   0   &  0   &   0   \cr
-D_2 &  I   &   0   &  0   &   0   \cr
 0   & -D_3 &   I   &  0   &   0   \cr
 0   &   0  &  -D_4 &  I   &   0   \cr
 0   &   0  &   0   & -D_5 &   I
} \in \mathbb{R}^{np \times np}$,
$T =
\pmatrix{
 0 & 0 & 0 & 0 & -D_1  \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0
} \in \mathbb{R}^{np \times np}$, 
$g =
\pmatrix{
f_1  \cr
f_2  \cr
f_3  \cr
f_4  \cr
f_5
}
\in \mathbb{R}^{np}$,
$v^{\{k\}} =
\pmatrix{
u^{\{(k-1)+1/5\}}  \cr
u^{\{(k-1)+2/5\}}  \cr
u^{\{(k-1)+3/5\}}  \cr
u^{\{(k-1)+4/5\}}  \cr
u^{\{k\}}
}
\in \mathbb{R}^{np}$,

In [49]:
# Construct S, T and g and solve
S= Smatrix_multiplicative(D,n,p,iprint)
T= Tmatrix_multiplicative(D,n,p,iprint)
g= gvector_multiplicative(f,n,p,iprint)

wstore= STiteration(S,T,g,n,p,K,iprint)
if iprint >= 4:
    print('wstore')
    print(wstore)

Print S matrix
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
Print T matrix
[[0.         0.         0.         ... 0.03907181 0.0323837  0.05211533]
 [0.         0.         0.         ... 0.0744902  0.02371058 0.05289307]
 [0.         0.         0.         ... 0.03577014 0.05314707 0.01841316]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
Print g vector
[[ 0.00310865]
 [ 0.00825975]
 [ 0.01783988]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.00427171]
 [ 0.01773143]
 [ 0.05238996]
 [ 0.04285054]
 [-0.00208064]
 [ 0.00108153]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

#### Perform $K$ iterations as single matrix solve

$K=6$ iterations of multiplicative Schwarz can be written as the $npK$-dimensional system of equations $Uz=h$, where

$U=
\pmatrix{
 S & 0 & 0 & 0 & 0 & 0   \cr
 T & S & 0 & 0 & 0 & 0   \cr
 0 & T & S & 0 & 0 & 0   \cr
 0 & 0 & T & S & 0 & 0   \cr
 0 & 0 & 0 & T & S & 0   \cr
 0 & 0 & 0 & 0 & T & S
}
$
,
$z=
\pmatrix{
v^{\{1\}}  \cr
v^{\{2\}}  \cr
v^{\{3\}}  \cr
v^{\{4\}}  \cr
v^{\{5\}}  \cr
v^{\{6\}}
}
$
,
$
h=
\pmatrix{
g  \cr
g  \cr
g  \cr
g  \cr
g  \cr
g
}
$
.

In [50]:
# Construct U and h and solve
U= Umatrix_multiplicative(S,T,n,p,K,iprint) 
h= hvector_multiplicative(g,n,p,K,iprint)

# Solve using U and h
zstore= Uhsolve_multiplicative(U,h,n,p,K,iprint);
if iprint >= 4:
    print('zstore')
    print(zstore)

h vector
[[ 0.00310865]
 [ 0.00825975]
 [ 0.01783988]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.00427171]
 [ 0.01773143]
 [ 0.05238996]
 [ 0.04285054]
 [-0.00208064]
 [ 0.00108153]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.04381268]
 [-0.00524394]
 [-0.00460419]
 [ 0.07103158]
 [ 0.04115988]
 [ 0.00632209]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.03913504]
 [ 0.01206122]
 [ 0.01482181]
 [ 0.05936004]
 [ 0.00310865]
 [ 0.00825975]
 [ 0.01783988]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.00427171]
 [ 0.01773143]
 [ 0.05238996]
 [ 0.04285054]
 [-0.00208064]
 

#### Compare Solutions from all three methods

In [51]:
solution_compare(xstore,vstore,wstore,zstore,iprint)

norm(exact - iterative) solutions    =  4.055123e-06
Difference between vstore and wstore =  5.890082e-17
Difference between wstore and zstore =  9.058118e-18
Difference between vstore and zstore =  5.857462e-17


### Add discretization error and compute error estimates and effectivity ratios

#### $K$  iterations computing subdomain contributions independently

In [52]:
# Set adjoint RHS
psi= np.ones((n,1))

# Perform D iteration with error: add random error to each subdomain solve
v_store,va_store,r_store= Diteration_approx(D,f,n,p,K,iprint,mpert,wpert)
if iprint >= 6:
    print(v_store)
    print(va_store)
    print(rstore)

Outer iteration  0
Outer iteration   0, Inner iteration   0
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 3.14370185e-03]
 [ 8.1285

#### Calculate the discretization error

In [53]:
# Perform adjoint solve 
phi_store= Diteration_adjoint(D,psi,n,p,K,iprint)

# Compute error estimate as (r,phi)
qoi_discretization_error_estimate= r_store.T @ phi_store

# Compute error and error in QoI directly
vesoln=  v_store[n*p*K + n*(p-1):n*p*(K+1),0:1]
vsoln=  va_store[n*p*K + n*(p-1):n*p*(K+1),0:1]
verror= vesoln - vsoln

qoi_discretization_error= psi.T @ verror

k, i, pindex, id1, id2: 3 4 0 132 144
Diteration_adjoint: special case 

k, i, pindex, id1, id2: 3 3 3 120 132
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+

#### Calculate the effectivity ratio

In [54]:
effectivity_ratio_disc = qoi_discretization_error_estimate / qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}')
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

QoI_discretization_error          =  1.255022e-04
QoI_discretization_error_estimate =  1.255022e-04
Effectivity ratio                 =         1.000


#### Calculate the total error

In [55]:
total_error= xstore - vsoln
qoi_total_error= psi.T @ total_error

phi= np.linalg.solve(M.T,psi)
r= y - M @ vsoln
qoi_total_error_estimate= r.T @ phi

#### Calculate the effectivity ratio

In [56]:
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   =  1.186189e-04
QoI_total_error_estimate          =  1.186189e-04
Effectivity ratio                 =         1.000


#### Calculate the iteration error

In [57]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               = -6.883287e-06


#### $K$ iterations as single matrix solve

In [58]:
# Perform U iteration with error: add random error to linear system solution
zexact,zapprox,zresid= Uhsolve_multiplicative_approx(U,h,n,p,K,iprint,mpert,wpert)

# Adjoint data (Adjoint system is of size npK)
psi_npK= np.zeros((n*p*K,1))
psi_npK[n*p*K - n:n*p*K,0]= psi.reshape(n)

#  Solve adjoint equation
phi_npK= np.linalg.solve(U.T,psi_npK)
if iprint >= 2:
    print(f'Norm of difference between phi_npK and phi_store = {np.linalg.norm(phi_npK-phi_store):13.6e}')

# Compute error estimate
qoi_discretization_error_estimate= phi_npK.T @ zresid

# Compute error and error in QoI directly
xsoln= zexact[n*p*K - n:n*p*K,0:1]
zsoln= zapprox[n*p*K - n:n*p*K,0:1]
zerror= xsoln-zsoln
qoi_discretization_error= psi.T @ zerror

[ 0.00310865  0.00825975  0.01783988  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.00310865  0.00423357  0.01751076  0.05223098  0.04278771 -0.00208609
  0.00106365  0.          0.          0.          0.          0.
  0.00310865  0.00423357  0.01751076  0.05223098  0.04099262 -0.00765256
 -0.00540388  0.06658251  0.03862675  0.00578671  0.          0.
  0.00310865  0.00423357  0.01751076  0.05223098  0.04099262 -0.00765256
 -0.00540388  0.06658251  0.03496864  0.00507412  0.00788476  0.05114969]
[-0.00420591 -0.00178986  0.01186696  0.05223098  0.04099262 -0.00765256
 -0.00540388  0.06658251  0.03496864  0.00507412  0.00788476  0.05114969
 -0.00420591 -0.00144977  0.01190354  0.04751062  0.03839566 -0.00771082
 -0.00611321  0.06658251  0.03496864  0.00507412  0.00788476  0.05114969
 -0.00420591 -0.00144977  0.01190354  0.04751062  0.03833974 -0.00784044
 -0.00623994  0.06670595  0.03603612  0.00616501  0.00788476  0.05114969
 -0.00420

#### Calculate the effectivity ratio

In [59]:
effectivity_ratio_disc= qoi_discretization_error_estimate/qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}' )
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

QoI_discretization_error          = -1.207370e-04
QoI_discretization_error_estimate = -1.207370e-04
Effectivity ratio                 =         1.000


#### Calculate the total error

In [60]:
total_error= xstore - zsoln
qoi_total_error= psi.T @ total_error

r= y - M @ zsoln
qoi_total_error_estimate= r.T @ phi

#### Calculate the effectivity ratio

In [61]:
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   = -1.276203e-04
QoI_total_error_estimate          = -1.276203e-04
Effectivity ratio                 =         1.000


#### Calculate iteration error

In [62]:
qoi_iteration_error= qoi_total_error - qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               = -6.883287e-06
